# Instagram Graph API

### Hashtag search and hashtag media fetching

In this notebook, instagram hashtag data is fetched with four main steps:
 - Create tables for stroing data
 - Get a list of # hashtag name
 - Using GET /ig_hashtag_search?user_id={user-id}&q={q} to aquire ig-hashtag-id
 - Using GET /{ig-hashtag-id}/{top_media}?user_id={user-id}&fields={fields} to obtain hashtag media metadata

Some tables and view are created beforehead for storing data in Postgres
 - Create Table `posts`: a list of hashtag media information contains nine fields, which are  `ig_id` `ig_hashtag_id`, `caption`, `query_type`, `media_type`, `permerlink`, `common_count`, `like_count`, `timestamp`
 - Create Table `tags`: A list of main hashtags where is marked is_main =1, and tag are hashtags that are used together with posts
 - Create Table `post_tags`: an indexing table that combine the posts_id and tags_id for data manipulation
 - Create View `tag_list`: to determin what hashtags are popular to used toghether with a list of predefined hashtags. Aggragating data based on table `posts`, `tags`, `post_tags` to get a list of hashtags that are used in top_media posts
 - Create View `tag_list_top_media`: a select statment is excuted to filter out query_type = top_media


### Loading Packges 

In [3]:
import requests
import re
import pandas as pd
import pprint
import json
import urllib.request 
import urllib.error
from time import gmtime, strftime
import time
import datetime
from time import sleep
import psycopg2
import config
import os.path

### Set database connection loading data to Postgres data lake

In [ ]:
# connect to postgres database
conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
cursor = conn.cursor()
conn.autocommit = True

'''
create `posts` table:
This is the main table that will use to generate hashtags from each posts 
'''

def createPostsTable():       
    cursor.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'posts'")
    if cursor.fetchone()[0]!= 1: #check if table `posts` is exist
        cursor.execute("""
        CREATE table "public"."posts" (
        "pos_id" SERIAL PRIMARY KEY not null,
        "ig_id" BIGINT null,
        "ig_hashtag_id" BIGINT null,
        "timestamp" INT null,
        "caption" TEXT null,
        "query_type" VARCHAR(128) not null,
        "media_type" TEXT null,
        "permalink_url" VARCHAR(128) UNIQUE not null,
        "comment_count" INT null,
        "like_count" INT null);
        """)

'''
create `tags` table:
This is the main table that will use to generate hashtags from each posts 
'''
        
def createTagsTable():
    cursor.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'tags'")
    if cursor.fetchone()[0] != 1:
        #print('create table')
        cursor.execute(f"""
        CREATE TABLE "public"."tags" (
          tag_id SERIAL PRIMARY KEY not null,
          ig_hashtag_id BIGINT DEFAULT NULL,
          tag VARCHAR(64) UNIQUE DEFAULT NULL,
          is_main SMALLINT DEFAULT NULL CHECK (is_main > 0));
          """)
        hashtags=['inlovewithswitzerland','switzerland','schweiz','suisse','zurich','bern','luzern','uri','schwyz','obwalden','nidwalden','glarus','zug','freiburg','solothurn','baselstadt','basellandschaft','schaffhausen','appenzellinnerrhoden','appenzellausserrhoden','stgallen','graubünden','aargau','thurgau','ticino','vaud','valais','neuchatel','geneve','jura']
        for hashtag in hashtags:
            cursor.execute(f"""INSERT INTO tags(tag, is_main) VALUES('{hashtag}',1) ON CONFLICT (tag) DO UPDATE SET tag = '{hashtag}', is_main = 1 """)
            

'''
create `tags_list` view:
the purpose of create view is to prevent dupilcation in the database
'''            
def createTagListView():
    table='tag_list'
    cursor.execute(f"""SELECT COUNT(*) FROM information_schema.tables WHERE table_name ='{table}'""")
    if cursor.fetchone()[0] != 1:
        cursor.execute(f"""CREATE VIEW tag_list AS
            SELECT
               tags.tag_id AS tag_id,
               tags.tag AS tag,count(post_tags.pos_id) AS post_count,
               posts.query_type
            FROM tags left join post_tags on tags.tag_id = post_tags.tag_id
            join posts on post_tags.pos_id = posts.pos_id
            group by posts.query_type,tags.tag_id
            order by post_count desc;""")

## create view tag_list_top_media based on query post = top_media         
def createTagListTopMediaView():
    table='tag_list_top_media'
    cursor.execute(f"""SELECT COUNT(*) FROM information_schema.tables WHERE table_name ='{table}'""")
    if cursor.fetchone()[0] != 1:
        cursor.execute(f"""CREATE VIEW tag_list_top_media AS
            SELECT
               tags.tag_id AS tag_id,
               tags.tag AS tag,count(post_tags.pos_id) AS post_count,
               posts.query_type
            FROM tags left join post_tags on tags.tag_id = post_tags.tag_id join posts on post_tags.pos_id = posts.pos_id
            WHERE posts.query_type = 'top_media' 
            group by posts.query_type,tags.tag_id
            order by post_count desc;""")
        
# fill in main hashtag in tag table, and marked as 1
def mainHashtag(category='hashtag'):
    #table='tags'
    cursor.execute(f"""SELECT ig_hashtag_id,tag FROM tags WHERE is_main = 1 """)
    rows = cursor.fetchall()
    hashtags=[]
    for row in rows:
        if category=='hashtag':
            hashtags.append(row[1])
        elif category=='all':
            hashtags.append([row[1],row[0]])
    return hashtags

createPostsTable()
createTagsTable() 
createTagListView() 
createTagListTopMediaView()


## make Api call to get ig-hashtag-id of each main hashtag in tags table
def getHashtagIds():
    hashtag_list = mainHashtag('all') # get the list of ig-hashtag-id from previous function mainHashtag()
    access_token="EAAIekqAtYJYBAD4XzhMop1kcG4Wolk2FZBgejbhfuOvgAMogKaPgjZBaaEPXQVPnOx0u8gmkYzInkE5gN6WheYPxdebqNvSDFyMKbffP5rh75ZBWgwdS0QenwJn42H601kHOOzQdvQDUJ8ObKuREdeidHrZCJhFbAAZAAygF0QgZDZD"
    domain='https://graph.facebook.com/v12.0'
    instagram_business_account='17841408501997863'
    conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
    cursor = conn.cursor()
    conn.autocommit = True
    for hashtag_infos in hashtag_list:
        hashtag = hashtag_infos[0]
        hashtag_encoded = urllib.parse.quote_plus(hashtag)
        # due to limit call for the api 'appenzellinnerrhoden' need to add hashtag list on 11/24
        #if hashtag == 'appenzellinnerrhoden': '17842294453030357' added on Dec 1, 2021
            #continue
        if hashtag_infos[1] == None or 1:
            url = f'{domain}/ig_hashtag_search?user_id={instagram_business_account}&q={hashtag_encoded}&access_token={access_token}'
            r = requests.get(url)
            data = r.json()
            row = []
            for index, item in enumerate(data['data'][0]):
                # print(data['data'][index]['id'])
                ig_hashtag_ids = data['data'][index]['id'] #parameter
                cursor.execute(f"""UPDATE "public"."tags" SET ig_hashtag_id= '{ig_hashtag_ids}' WHERE tag= '{hashtag}' """)                    
            print(ig_hashtag_ids)

getHashtagIds()


### Get Hashtage media using IG Hashtag

The script below contains four steps from fetching data through api call and till inserting data to database
parameters for accessing endpoints need to be defined in the begining. 

Since top_mdeia & recent_media are updated frequenty, airflow pipline is set for excuting data fetching tasks.

 - 1. for loop: looing through ig_hashtag_list to get json file and saved with ig_hashtag_id
 - 2. get field value using for loop with `enumerate` to aquire row value in accordance with coloum names and dump to `posts` which is created in the previous section
 - 3. check if `pos_id` is exist in the `posts` table, filter out hashtags appear in the caption section, than insert to `tags` table and insert `tag_id` for new generate `tag`
 - 4. Insert `tag_id`to `post_tags` so as to perform `tag_list` and `tag_list_top_media` views to get ranked of hashtag list

In [4]:
def callApi():
    instagram_business_account = "instagram_business_account"
    access_token="your access_token "
    domain="https://graph.facebook.com/v12.0"
    ig_hashtag_ids = ['17843695108061880','17841562486127746','17843781943053326','17843737147012077','17841563617103035','17843770291063460','17843731063037362','17843674843009135','17843766175034928','17841563632129050','17841563290127743','17841563314095275','17843857468005360','17843794165015026','17841589219074534','17843715058054899','17841594358073046','17841562702112282','17841563410095061','17841549814118642','17843714212014818','17841514093097929','17843668015049047','17843721961051022','17841564079084570','17843678611020704','17843678038040490','17841562906121517','17843977789033663','17842294453030357']
    query_types=['recent_media','top_media']
    fetch_from_api=1 # activate api call, incase got block 

    #connect to postgres
    conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
    cursor = conn.cursor()
    conn.autocommit = True
    now = time.time()
    timeToLive = 3600 # 1 hour

    # looping through a list of hashtags in `ig_hashtag_ids` 
    for ig_hashtag_id in ig_hashtag_ids:
        for query_type in query_types:
            filename=query_type+'_'+ig_hashtag_id+'.json' #save file name to chech if all data are load
            print(filename)
            if fetch_from_api: # if fetch_from_api = 1 means is fetching data
                url = f'{domain}/{ig_hashtag_id}/{query_type}?user_id={instagram_business_account}&fields=id,caption,media_type,media_url,permalink,comments_count,like_count,time,timestamp&access_token={access_token}'
                print(url)
                #exit()
                if os.path.isfile(filename) and os.path.getmtime(filename)+timeToLive<now:
                    print('remove file:'+filename) 
                    os.remove(filename)
                if os.path.isfile(filename): # check if data can be loaded locally 
                    print('file exists:'+filename) 
                    with open(filename, 'r') as json_file:
                        data=json_object = json.load(json_file)
                else:
                    print('file does not exist:'+filename) # debug problem hashtag entry
                    print('')
                    #continue

                    with urllib.request.urlopen(url) as url:
                        data = json.loads(url.read().decode())
                    with open(filename, 'w') as f:
                        json.dump(data, f)
                    time.sleep(2) # deactivate api call to avoid getting blocked
            else:
                # get data from file
                if os.path.isfile(filename): # check if data can be loaded locally
                    with open(filename, 'r') as json_file:
                        data=json_object = json.load(json_file)
                else:
                    print('file does not exist:'+filename) # debug problem hashtag entry
            row=[]
            #print(data['data'])
            for index,item in enumerate(data['data']):
                #print(index)
                #print(data['data'][index])
                like_count=data['data'][index].get('like_count',0)
                caption=data['data'][index].get('caption','')
                timestamp=int(time.mktime(datetime.datetime.strptime(data['data'][index]['timestamp'], "%Y-%m-%dT%H:%M:%S%z").timetuple()))
                # insert post into posts
                sql = "INSERT INTO posts (ig_id,ig_hashtag_id,timestamp,caption,query_type,media_type,permalink_url,comment_count,like_count) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (permalink_url) DO NOTHING RETURNING pos_id;"
                row=[
                    data['data'][index]['id'],
                    ig_hashtag_id,
                    timestamp,
                    caption,
                    query_type,
                    data['data'][index]['media_type'],
                    data['data'][index]['permalink'],
                    data['data'][index]['comments_count'],
                    like_count
                ]
                cursor.execute(sql, tuple(row))
                result=cursor.fetchone()
                pos_id=0 
                if result and result[0]:
                    pos_id=result[0]
                if pos_id:
                    hashtags = re.findall('[#]([^#\s,.]+)',caption) # using regular expression to seprate hashtag in capition field
                    print(hashtags)
                    row2=[]
                    for hashtag in hashtags:
                        sql = "SELECT tag_id from public.tags WHERE tag = %s"
                        row2=[hashtag]
                        cursor.execute(sql, tuple(row2))
                        result=cursor.fetchone()
                        tag_id=0
                        if result and result[0]:
                            # get tag id
                            tag_id=result[0]
                        else:
                            # insert new tag into tags
                            sql = "INSERT INTO public.tags (tag) VALUES (%s) RETURNING tag_id"
                            row3=[]
                            row3=[hashtag]
                            result=cursor.execute(sql, tuple(row3))
                            if result and result[0]:
                                tag_id=result[0]

                        if tag_id:
                            #connect tag to post
                            sql = "INSERT INTO public.post_tags (pos_id,tag_id) VALUES (%s,%s)"
                            row4=[]
                            row4=[pos_id,tag_id]
                            cursor.execute(sql, tuple(row4))
                            #postag_id=cursor.lastrowid
callApi()

recent_media_17843695108061880.json
https://graph.facebook.com/v12.0/17843695108061880/recent_media?user_id=17841408501997863&fields=id,caption,media_type,media_url,permalink,comments_count,like_count,time,timestamp&access_token=EAAIekqAtYJYBAD4XzhMop1kcG4Wolk2FZBgejbhfuOvgAMogKaPgjZBaaEPXQVPnOx0u8gmkYzInkE5gN6WheYPxdebqNvSDFyMKbffP5rh75ZBWgwdS0QenwJn42H601kHOOzQdvQDUJ8ObKuREdeidHrZCJhFbAAZAAygF0QgZDZD
remove file:recent_media_17843695108061880.json
file does not exist:recent_media_17843695108061880.json

['grimselpass', 'inlovewithswitzerland', 'adventurestartshere', 'climbingnation', 'rockclimbing', 'climbinginspiration', 'climax_magazine', 'climbing_worldwide', 'climbingnation', 'liveclimbrepeat', 'climbinglife', 'isofunus', 'climbingisbliss']
['aechtschwyz', 'mytheregion', 'hochybrig', 'awesome_earthpix', 'photooftheday', 'picoftheday', 'outdoorlover', 'getoutdoors', 'neverstopexploring', 'instagood', 'landscapephotography', 'landschaftsfotografie', 'visitswitzerland', 'inlovewiths

top_media_17843695108061880.json
https://graph.facebook.com/v12.0/17843695108061880/top_media?user_id=17841408501997863&fields=id,caption,media_type,media_url,permalink,comments_count,like_count,time,timestamp&access_token=EAAIekqAtYJYBAD4XzhMop1kcG4Wolk2FZBgejbhfuOvgAMogKaPgjZBaaEPXQVPnOx0u8gmkYzInkE5gN6WheYPxdebqNvSDFyMKbffP5rh75ZBWgwdS0QenwJn42H601kHOOzQdvQDUJ8ObKuREdeidHrZCJhFbAAZAAygF0QgZDZD
remove file:top_media_17843695108061880.json
file does not exist:top_media_17843695108061880.json

['planetwanderlust']
['سويسرا', 'النمسا', 'انترلاكن', 'طبيعة', 'switzerland', 'swiss', 'schweiz', 'suisse', 'svizzera', 'suiza', 'Швейцария', '스위스', 'isviçre', 'interlaken', 'nature', 'naturaleza', 'natureza', 'природа', 'natur', 'landscape', 'spring', 'countryside', 'swissalps', 'alps', 'alpen', 'mountains', 'swissmountains', 'inlovewithswitzerland', 'myswitzerland', 'switzerlandpictures']
['fog', 'fogessentials', 'foggy', 'foggymorning', 'mist', 'mistyfoggymilkymoody', 'visitswitzerland', 'swit

['naturaltravel', 'Suisse', 'Amazingswitzerland', 'Swissalps', 'Schweiz', 'Suiza', 'suica', 'switzerland', 'ig_switzerland', 'svizzera', 'Hiking', 'alps', 'switzerland', 'myswitzerland', 'visitswitzerland', 'inlovewithswitzerland', 'swiss', 'swissmountains', 'nature', 'loveswitzerland', 'iloveswitzerland', 'bestplacestogo', 'earthfocus', 'mountains', 'hikingadventures', 'outplanetdaily', 'isviçre']
['swisshotel', 'swisshotels', 'schweizerhotel', 'whitepod', 'ecoluxury', 'switzerlandwonderland', 'switzerlandhotels', 'valais', 'wallis', 'blickheimat', 'amazingswitzerland', 'hotelsandresorts', 'visitswitzerland', 'inlovewithswitzerland', 'epicswitzerland', 'epic_landscape', 'winterstyle', 'swissinfluencers', 'kinginfluencer', 'swisblogger', 'familyblog', 'takeyourkidseverywhere', 'adventurevisual', 'adventureanywhere', 'adventureculture', 'thebestofswitzerland', 'super_switzerland', 'schweiztourismus', 'switzerlandgoodvibes', 'alpenliebe_official']
recent_media_17841562486127746.json
http

['scuol', 'schnee', 'bergwelt', 'agata', 'agatascuol', 'sanjong', 'pferdekutsche', 'pferdeschlitten', 'pferdeschlittenfahrt', 'naturephotography', 'swissmountains', 'swissalps', 'engadin', 'engadina', 'unterengadin', 'schneezauber', 'schneelandschaft', 'engiadina', 'engiadinabassa', 'schuls', 'bündnerland', 'graubünden', 'sonnenuntergang', 'schweiz', 'switzerland', 'winterwonderland', 'ardez', 'ftan', 'guarda', 'sent']
['shooting', 'schweiz', 'tour', 'model', 'business', 'photooftheday', 'photography', 'ootd', 'winter', 'snow']
['มีทองท่วมหัวหรือจะสู้มีผัววิดวะคอม']
['geneve', 'suisse', 'jonctiongeneve', 'geneva', 'genf', 'switzerland', 'myswitzerland', 'schweiz', 'paisaje', 'landscapephotography', 'landscape', 'paysage', 'naturaleza', 'nature', 'riviere', 'promenade', 'travel', 'arquitectura', 'architecture', 'architecturephotography', 'rhone', 'rodano', 'withgalaxy', 'city', 'ville', 'ciudad', 'nature', 'natureenville', 'naturaleza', 'naturephotography', 'cityscape']
top_media_178415

['moods_in_frame', 'earth_shotz', 'eclectic_shotz', 'depthsofearth', 'myswitzerland', 'thegreatplanet', 'wildernessculture', 'swiss', 'earthhotspots', 'earthoutdoors', 'uniladadventure', 'roamtheplanet', 'mountainspirit', 'swissxplorers', 'schweiz', 'shooters', 'earthfocus', 'alpenglow', 'wekeepmoments', 'toms_week', 'voyaged', 'switzerland_vacations']
recent_media_17843781943053326.json
https://graph.facebook.com/v12.0/17843781943053326/recent_media?user_id=17841408501997863&fields=id,caption,media_type,media_url,permalink,comments_count,like_count,time,timestamp&access_token=EAAIekqAtYJYBAD4XzhMop1kcG4Wolk2FZBgejbhfuOvgAMogKaPgjZBaaEPXQVPnOx0u8gmkYzInkE5gN6WheYPxdebqNvSDFyMKbffP5rh75ZBWgwdS0QenwJn42H601kHOOzQdvQDUJ8ObKuREdeidHrZCJhFbAAZAAygF0QgZDZD
remove file:recent_media_17843781943053326.json
file does not exist:recent_media_17843781943053326.json

['teamdropix', 'austriatrip', 'luzern', 'loveaustria', 'salzburg', 'denamark', 'igaustria', 'austriatoday', 'germany', 'switzerland', 

...

['art', 'artwork', 'artoftheday', 'artwork', 'artlover', 'artcollector', 'kunst', 'kunstwerk', 'malerei', 'painting', 'aquarelle', 'watercolor', 'watercolour', 'illustration', 'humor', 'karikatur', 'caricature', 'ventilation', 'ventilator', 'lüftung', 'wind', 'windisch', 'renewalker_riisundboor', 'berikon', 'aargau', 'swissartist', 'schweizerkünstler']
['rheinfeldenschweiz', 'rheinfelden', 'adventsfunkeln', 'aargaueraltstädte', 'rheinliebe', 'weihnachtsbeleuchtung', 'advent', 'aargau', 'aargautourismus', 'citylife', 'swisscities', 'adventsfenster']
top_media_17841564079084570.json
https://graph.facebook.com/v12.0/17841564079084570/top_media?user_id=17841408501997863&fields=id,caption,media_type,media_url,permalink,comments_count,like_count,time,timestamp&access_token=EAAIekqAtYJYBAD4XzhMop1kcG4Wolk2FZBgejbhfuOvgAMogKaPgjZBaaEPXQVPnOx0u8gmkYzInkE5gN6WheYPxdebqNvSDFyMKbffP5rh75ZBWgwdS0QenwJn42H601kHOOzQdvQDUJ8ObKuREdeidHrZCJhFbAAZAAygF0QgZDZD
remove file:top_media_17841564079084570.json


['facehype', 'facehype', 'face', 'hype', 'faceit', 'konstanz', 'jawlinefiller', 'juvederm', 'konstanz', 'zurich', 'thurgau', 'kreuzlingen', 'winterthur', 'stgallen', 'schaffhausen', 'lippenaufspritzen', 'lippenunterspritzung', 'lippenaufbau', 'lippenaufspritzen', 'hyaluron', 'botox', 'filler', 'skincare', 'antiaging', 'aesthetics', 'loveyourself', 'womanstyle', 'menstyle', 'blogger']
top_media_17843678611020704.json
https://graph.facebook.com/v12.0/17843678611020704/top_media?user_id=17841408501997863&fields=id,caption,media_type,media_url,permalink,comments_count,like_count,time,timestamp&access_token=EAAIekqAtYJYBAD4XzhMop1kcG4Wolk2FZBgejbhfuOvgAMogKaPgjZBaaEPXQVPnOx0u8gmkYzInkE5gN6WheYPxdebqNvSDFyMKbffP5rh75ZBWgwdS0QenwJn42H601kHOOzQdvQDUJ8ObKuREdeidHrZCJhFbAAZAAygF0QgZDZD
remove file:top_media_17843678611020704.json
file does not exist:top_media_17843678611020704.json

[]
['autumnshooting', 'autumnvibes🍁', 'swissautumn', 'thurgau', 'schweiz🇨🇭', 'sonyswiss', 'sonydeutschland', 'sony

recent_media_17841562906121517.json
https://graph.facebook.com/v12.0/17841562906121517/recent_media?user_id=17841408501997863&fields=id,caption,media_type,media_url,permalink,comments_count,like_count,time,timestamp&access_token=EAAIekqAtYJYBAD4XzhMop1kcG4Wolk2FZBgejbhfuOvgAMogKaPgjZBaaEPXQVPnOx0u8gmkYzInkE5gN6WheYPxdebqNvSDFyMKbffP5rh75ZBWgwdS0QenwJn42H601kHOOzQdvQDUJ8ObKuREdeidHrZCJhFbAAZAAygF0QgZDZD
remove file:recent_media_17841562906121517.json
file does not exist:recent_media_17841562906121517.json

['balayage', 'hair', 'haircolor', 'behindthechair', 'hairstyle', 'hairstylist', 'highlights', 'olaplex', 'hairgoals', 'hairstyles', 'modernsalon', 'hairdresser', 'babylights', 'balayagehair', 'beauty', 'balayageombre', 'balayagehighlights', 'redken', 'longhair', 'blondebalayage', 'ombrehair', 'hairinspo', 'salon', 'hairtransformation', 'neuchatel', 'neuch', 'lausanne', 'bienne', 'bern', 'suisse']
['suisse', 'suisse🇨🇭', 'neuchatel', 'neuchâtel', 'montagne', 'zemmourpresident', 'zemmour

['law', 'mindermeinung', 'studentproblems', 'herrschendemindermeinung', 'institut', 'gesetz', 'universität', 'studentlife', 'studylawtheysaid', 'lawstudent', 'meme', 'studentmemes', 'university', 'fun', 'funny', 'like', 'memeoftheday', 'jus', 'jura', 'jusmemes', 'juramemes']
['hbbfc', 'gasballoon', 'gasballooning', 'jura', 'juracoffee', 'juracoffeemachine', 'oldschoolballooning', 'silentflight', 'stadwil', 'bronschhofen', 'switzerland', 'schweiz', 'swissballoons', 'ballonfahren', 'ballonfahrt', 'analogphotography']
['jura', 'juramemes', 'jurameme', 'jurastudent', 'jurastudium', 'uni', 'student', 'universität', 'strafrecht', 'bibliothek', 'anwalt', 'rechtsanwalt', 'staatsanwalt', 'stgb', 'bgb', 'grundgesetz', 'gesetz', 'law', 'examen', 'examensvorbereitung', 'zivilrecht', 'huberlin', 'fuberlin', 'unipotsdam', 'viadrina', 'lmumünchen', 'bucerius', 'unitübingen', 'lawyer', 'lawlife']
['oyonnax', 'dépanneur', 'sav', 'portail', 'ain', 'jura', 'hautjura', 'renovationmaison', 'travauxmaison',

['jura', 'cascade', 'montagnesdujura', 'bfctravel', 'fairy', 'waterfall', 'beautifulfrance', 'naturelovers', 'lovescape', 'bfc_focus_on']
['jura', 'juratourisme', 'montagnesdujura', 'sapins', 'atmosphere', 'beautifuldestinations', 'bourgognefranchecomte', 'paysagesmagnifiques', 'leroutard', 'lesfrancaisvoyagent', 'francetourisme', 'francetrip', 'randonnée', 'lens_loves_nature', 'nomadict', 'paysagephotography', 'winterwonderland', 'baladeenforet', 'divine_forest', 'merveillesdefrance', 'folkvibe', 'winterlandscape', 'minimalism', 'partageducoq', 'francetourisme', 'travelgram', 'france4dreams', 'francemontagnes', 'paysagedefrance', 'nature_focus_on']
recent_media_17842294453030357.json
https://graph.facebook.com/v12.0/17842294453030357/recent_media?user_id=17841408501997863&fields=id,caption,media_type,media_url,permalink,comments_count,like_count,time,timestamp&access_token=EAAIekqAtYJYBAD4XzhMop1kcG4Wolk2FZBgejbhfuOvgAMogKaPgjZBaaEPXQVPnOx0u8gmkYzInkE5gN6WheYPxdebqNvSDFyMKbffP5rh75ZBW

### Get a view of the most popular hashtags using in the top_media 

Here we can see,  **#Switzerland** is topped the rank of tag list, then followed by **#schweiz**, **#myswitzerland**, **#visitswitzerland**, and the main hashtag **#inlovewithswitzerland** from MySwitzerland. 
Nevertheless, the result cannot tell us what locations or attraction people have been visiting the most in Switzerland, hence, additional data source can support us to identify locations is needed, so that we can filter out wording tags. Additional data source will be add in the second half of the semester project, and will be presented in the final submitting.

In [ ]:
import pandas as pd
conn = psycopg2.connect(
    host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",
    database="datalake1",user="MVision",password="MVision123")
cursor = conn.cursor()
conn.autocommit = True

# tag_list_top_media
mydata= pd.read_sql(sql = "SELECT * FROM tag_list_top_media", con = conn);
df= pd.DataFrame(mydata)
print(df)


In [ ]:
# tag_list 
mydata2= pd.read_sql(sql = "SELECT * FROM tag_list", con = conn);
df2= pd.DataFrame(mydata2)
print(df2)